In [2]:
import pandas as pd
import altair as alt
import numpy as np

# Load the data (adjust path if needed)
df = pd.read_csv('mosquito_trap_results.csv')

In [5]:
# Handle missing values
df = df.dropna(subset=['SPECIES', 'RESULT', 'WEEK', 'SEASON YEAR'])

# Standardize RESULT column
df['RESULT'] = df['RESULT'].str.strip().str.upper()

# Create binary WNV positive indicator
df['WNV_POSITIVE'] = (df['RESULT'] == 'POSITIVE').astype(int)

# Convert TEST DATE to datetime
df['TEST DATE'] = pd.to_datetime(df['TEST DATE'], errors='coerce')

# Check your data
print(f"Total rows after cleaning: {len(df)}")
print(f"Year range: {df['SEASON YEAR'].min()} - {df['SEASON YEAR'].max()}")
print(f"Positive cases: {df['WNV_POSITIVE'].sum()}")
print(f"Total mosquitoes: {df['NUMBER OF MOSQUITOES'].sum()}")


Total rows after cleaning: 38215
Year range: 2007 - 2025
Positive cases: 3708
Total mosquitoes: 493953


/var/folders/xb/6v4wqmd106n911j8xnvpy0h40000gn/T/ipykernel_15942/1894728297.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['RESULT'] = df['RESULT'].str.strip().str.upper()
/var/folders/xb/6v4wqmd106n911j8xnvpy0h40000gn/T/ipykernel_15942/1894728297.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['WNV_POSITIVE'] = (df['RESULT'] == 'POSITIVE').astype(int)
/var/folders/xb/6v4wqmd106n911j8xnvpy0h40000gn/T/ipykernel_15942/1894728297.py:11: UserWarning: Could not infer format, so each element will b

In [6]:
# Create species summary
species_summary = df.groupby(['SPECIES', 'SEASON YEAR']).agg({
    'TEST ID': 'count',
    'WNV_POSITIVE': 'sum',
    'NUMBER OF MOSQUITOES': 'sum'
}).reset_index()

species_summary['POSITIVE_RATE'] = (
    species_summary['WNV_POSITIVE'] / species_summary['TEST ID'] * 100
)

species_summary.columns = ['Species', 'Year', 'Total_Tests', 
                            'Positive_Tests', 'Total_Mosquitoes', 'Positive_Rate']

           Species  Year  Total_Tests  Positive_Tests  Total_Mosquitoes  \
0  CULEX ERRATICUS  2013            1               0                 7   
1  CULEX ERRATICUS  2014            7               0                13   
2  CULEX ERRATICUS  2015            4               0                18   
3  CULEX ERRATICUS  2017            4               0                 6   
4  CULEX ERRATICUS  2020            1               0                 2   
5  CULEX ERRATICUS  2025            1               0                 1   
6    CULEX PIPIENS  2007         1562             145             35147   
7    CULEX PIPIENS  2008          307               6              1051   
8    CULEX PIPIENS  2009          371               0               867   
9    CULEX PIPIENS  2010          417               7              1460   

   Positive_Rate  
0       0.000000  
1       0.000000  
2       0.000000  
3       0.000000  
4       0.000000  
5       0.000000  
6       9.282971  
7       1.954397  
8  

In [28]:
labels = sorted(species_summary['Year'].unique().tolist())
species_order = sorted(species_summary['Species'].unique().tolist())

In [29]:
import altair as alt

# Create a dropdown for year selection
year_dropdown = alt.binding_select(
    options=[None] + labels,
    labels= ['All'] + [str(l) for l in labels],
    name='Select Year: '
)
year_selection = alt.selection_point(
    fields=['Year'],
    bind=year_dropdown,
    value=[{'Year': species_summary['Year'].max()}]  # Default to most recent year
)

# Base chart
base = alt.Chart(species_summary).add_params(
    year_selection
).transform_filter(
    year_selection
)

# Grouped bar chart: Total tests and positive tests
bar_chart = base.mark_bar().encode(
    x=alt.X('Species:N', 
            sort=species_order,
            scale=alt.Scale(domain=species_order),
            axis=alt.Axis(labelAngle=-45, title='Mosquito Species')),
    y=alt.Y('Total_Tests:Q', 
            title='Number of Tests'),
    color=alt.Color('Species:N', legend=None),
    tooltip=[
        alt.Tooltip('Species:N', title='Species'),
        alt.Tooltip('Year:Q', title='Year'),
        alt.Tooltip('Total_Tests:Q', title='Total Tests', format=','),
        alt.Tooltip('Positive_Tests:Q', title='Positive Tests', format=','),
        alt.Tooltip('Positive_Rate:Q', title='Positive Rate (%)', format='.2f'),
        alt.Tooltip('Total_Mosquitoes:Q', title='Total Mosquitoes', format=',')
    ]
).properties(
    width=600,
    height=400,
    title='Mosquito Species Testing Summary by Year'
)

# Overlay positive tests as a different mark
positive_bar = base.mark_bar(opacity=0.7).encode(
    x=alt.X('Species:N', sort=species_order, scale=alt.Scale(domain=species_order)),
    y=alt.Y('Positive_Tests:Q'),
    color=alt.value('red')
)

# Combine
species_viz = (bar_chart + positive_bar).configure_axis(
    labelFontSize=12,
    titleFontSize=14
).configure_title(
    fontSize=16,
    anchor='start'
)

# Save
species_viz.save('species_comparison.html')
print("Species visualization saved!")
species_viz.show()

Species visualization saved!


alt.LayerChart(...)

In [16]:
# Aggregate by week and year
weekly_data = df.groupby(['SEASON YEAR', 'WEEK']).agg({
    'WNV_POSITIVE': 'sum',
    'TEST ID': 'count',
    'NUMBER OF MOSQUITOES': 'sum'
}).reset_index()

weekly_data.columns = ['year', 'week', 'positive_cases', 'total_tests', 'total_mosquitoes']

# Calculate positive rate
weekly_data['positive_rate'] = (
    weekly_data['positive_cases'] / weekly_data['total_tests'] * 100
).fillna(0)

# Save to CURRENT directory (no '../data/' path)
weekly_data.to_csv('weekly_aggregated_data.csv', index=False)
print("✓ Weekly data saved to current directory!")
print(f"\nData preview:")
print(weekly_data.head(20))

✓ Weekly data saved to current directory!

Data preview:
    year  week  positive_cases  total_tests  total_mosquitoes  positive_rate
0   2007    21               0           25                40       0.000000
1   2007    22               0           59               126       0.000000
2   2007    23               0           46                85       0.000000
3   2007    24               0           70               216       0.000000
4   2007    25               0           74               240       0.000000
5   2007    26               1          127               633       0.787402
6   2007    27               0          152              1103       0.000000
7   2007    28               5          306              5237       1.633987
8   2007    29              13          324              5989       4.012346
9   2007    30               8          285              4003       2.807018
10  2007    31              20          303              6473       6.600660
11  2007    32     